In [10]:
from docx import Document
import pandas as pd
import re
import pickle
import os
from langchain.vectorstores import FAISS
from langchain.schema import Document as Doc
from langchain_openai import OpenAIEmbeddings
import shutil

In [5]:
def is_chapter_number(text):
    """
    Checks if the text is a single Hebrew letter (used as a chapter marker).
    """
    return re.fullmatch(r'[א-ת]', text.strip())

def docx_split_by_structured_titles(docx_path):
    """
    Splits a .docx file into structured documents based on Hebrew chapter markers.

    Each chapter starts with a single Hebrew letter (e.g., א) followed by a title line.
    The function groups the subsequent paragraphs as the chapter body, until the next chapter begins.

    Args:
        docx_path (str): Path to the .docx file.
        start_index (int): Optional index to start counting from (not currently used here).

    Returns:
        pd.DataFrame: A dataframe with structured documents including 'headline', 'document', 'source', and 'book_id'.
    """
    doc = Document(docx_path)
    documents = []
    current_title = None
    current_body = []
    skip_next = False  # Used to skip the paragraph immediately after a chapter marker (it's the title)

    for i in range(len(doc.paragraphs) - 1):
        para = doc.paragraphs[i]
        next_para = doc.paragraphs[i + 1]
        text = para.text.strip()
        next_text = next_para.text.strip()

        if skip_next:
            skip_next = False
            continue

        # Detect new chapter: current paragraph is a single Hebrew letter and next paragraph is non-empty (title)
        if is_chapter_number(text) and next_text:
            # Save the previous chapter if it exists
            if current_title and current_body:
                documents.append({
                    "headline": re.sub(r"^[א-ת]\s*-\s*", "", current_title),
                    "document": "\n".join(current_body),
                    "source": "DBGH",
                    "book_id": "memories"
                })
                current_body = []

            # Start a new chapter title
            current_title = f"{text} - {next_text}"
            skip_next = True
        else:
            if text:
                current_body.append(text)

    # Add the last chapter if needed
    if current_title and current_body:
        documents.append({
            "headline": re.sub(r"^[א-ת]\s*-\s*", "", current_title),
            "document": "\n".join(current_body),
            "source": "DBGH",
            "book_id": "memories"
        })
    # Return the structured data as a DataFrame
    df = pd.DataFrame(documents)
    return df


df = docx_split_by_structured_titles('./data/memories.docx')
print(df[['headline', "document"]].head())
df.to_csv('./data/memories.csv', index=False, encoding='utf-8-sig')

                          headline  \
0            ילדות ונעורים בפלונסק   
1         עלייתי לארץ. מכתבים לאבא   
2  חיי בסג'רה – עבודה ושמירה עברית   
3  חללי סג'רה – עבודתי בזכרון-יעקב   
4         מחקלאות לעתונות פועלים –   

                                            document  
0  נולדתי ביום י"ז בתשרי תרמ"ז (16.10.1886) בעייר...  
1  ארבעתנו, שושקה, אשתו של שמחה אייזיק, ובתה רחל ...  
2  אחרי יהודה היתה לי סג'רה כמעט מה שהיתה לי פתח-...  
3  סג'רה שהיתה ראשונה לשמירה עברית היתה גם ראשונה...  
4  מעתונות לפוליטיקה ולאוניברסיטה התורכית\nעבדתי ...  


In [6]:
# The same text pre-processing steps used for the initial dataset
def remove_html_tags(text):
    """Remove HTML tags."""
    text = re.sub(r'<[^<]+?>', '', text)
    text = re.sub(r'http\S+|www\S+', '', text)
    return text

def remove_special_characters(text):
    text = re.sub(r'[^A-Za-z0-9\s\.,;:\?\'\"\!\-\u0590-\u05FF]', '', text)
    return text.strip()

def remove_repeated_substrings(text):
    text = re.sub(r'([.?!,;:—"])\1+', r'\1', text)
    return text.strip()

def remove_extra_spaces(text):
    text = re.sub(r'\n\s*\n', '\n\n', text)
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

def preprocess_text(text):
    # Remove HTML tags
    text = remove_html_tags(text)
    
    # Remove special characters
    text = remove_special_characters(text)

    # Remove repeated substrings like dots
    text = remove_repeated_substrings(text)

    # Remove extra spaces between lines and within lines
    text = remove_extra_spaces(text)

    return text.strip()

In [7]:
# Apply preprocessing
df.loc[:, 'document_pp'] = df['document'].apply(preprocess_text)

In [8]:
df.head()

,headline,document,source,book_id,document_pp
0,ילדות ונעורים בפלונסק,"נולדתי ביום י""ז בתשרי תרמ""ז (16.10.1886) בעייר...",DBGH,memories,"נולדתי ביום י""ז בתשרי תרמ""ז 16.10.1886 בעיירה ..."
1,עלייתי לארץ. מכתבים לאבא,"ארבעתנו, שושקה, אשתו של שמחה אייזיק, ובתה רחל ...",DBGH,memories,"ארבעתנו, שושקה, אשתו של שמחה אייזיק, ובתה רחל ..."
2,חיי בסג'רה – עבודה ושמירה עברית,אחרי יהודה היתה לי סג'רה כמעט מה שהיתה לי פתח-...,DBGH,memories,אחרי יהודה היתה לי סג'רה כמעט מה שהיתה לי פתח-...
3,חללי סג'רה – עבודתי בזכרון-יעקב,סג'רה שהיתה ראשונה לשמירה עברית היתה גם ראשונה...,DBGH,memories,סג'רה שהיתה ראשונה לשמירה עברית היתה גם ראשונה...
4,מחקלאות לעתונות פועלים –,מעתונות לפוליטיקה ולאוניברסיטה התורכית\nעבדתי ...,DBGH,memories,מעתונות לפוליטיקה ולאוניברסיטה התורכית עבדתי ב...


In [11]:
# Convert dataframe rows to LangChain Documents
docs = [
    Doc(page_content=row['document_pp'], metadata={"book_id": row['book_id'], "headline": row["headline"] , "source": "DBGH"})
    for idx, row in df.iterrows()
]

In [13]:
def split_text_by_words(text, max_words, overlap_words):
    # Split the text into words
    words = text.split()

    chunks = []
    start_idx = 0
    while start_idx < len(words):
        end_idx = min(start_idx + max_words, len(words))
        chunk = words[start_idx:end_idx]

        # Join the words back into text
        chunk_text = ' '.join(chunk)
        chunks.append(chunk_text)

        # Update the starting position with overlap
        start_idx += max_words - overlap_words

    return chunks

In [15]:
chunks = []
idx = 21529

for doc in docs:
    temp_chunks = split_text_by_words(
        doc.page_content, max_words=260, overlap_words=35
    )
    
    for chunk in temp_chunks:
        new_metadata = dict(doc.metadata) 
        new_metadata['idx'] = idx 
        chunks.append(Doc(page_content=chunk, metadata=new_metadata))
        idx += 1

In [16]:
print(len(chunks))

111


In [17]:
with open('./data/chunks.pkl', 'rb') as f:
    initial_chunks = pickle.load(f)

In [18]:
print(len(initial_chunks))

21529


In [19]:
DBGH_chunks = initial_chunks + chunks
print(len(DBGH_chunks))
with open('./data/DBGH_chunks.pkl', 'wb') as f:
    pickle.dump(DBGH_chunks, f)

21640


In [21]:
DBGH_chunks[21527:21530]

[Document(metadata={'book_id': 88632, 'headline': 'לקראת "יציאת אירופה" - מאמר מתוך סדרת מאמרים: בדרך לצבא ולמדינת ישראל', 'source': 'DBGH', 'idx': 21527}, page_content='תהיה פחות מדולדלת משאר המעצמות הנלחמות. רווחת אירופה תהיה תלויה במידה רבה בעזרתה הכלכלית של ארצות-הברית. ושמירת השלום, שכנראה תהיה הדאגה הראשית של על אהדת בריטניה לעצמאות סוריה. המדינות המנצחות, תהיה תלויה יותר מתמיד בהשתתפותה האקטיבית של אמריקה בסידורים החדשים שלאחר המלחמה. אבל מה שלא יהיה מעמדה של אמריקה בעניני העולם - בענינים שלנו היא בוודאי תהא מכרעת. באמריקה יהיה יותר קל לרכוש דעת-הקהל לפתרון רדיקלי ומכסימלי של הבעיה היהודית בארץישראל, מאשר באנגליה. אמריקה חופשית מהתסביך הערביהמוסלימי, ואם שירותה הקונסולרי במזרח התיכון עלול אולי להיות שותף לתפיסת הפקידות הבריטית במזרח התיכון, הרי השפעת שירות זה רחוקה מזו של משרדי החוץ והמושבות באנגליה. אמריקה היא בהרבה פחות מעוניינת בארץ-ישראל מאנגליה, ולכן היא מוכשרה לנקוט עמדה יותר אובייקטיבית ובלתי-משוחדת. יש לזכור גם זאת: יש בה עדה יהודית גדולה - הגדולה בעולם - שהיא מתעניינת ב

In [ ]:
api_key = "your-api-key"
os.environ["OPENAI_API_KEY"] = api_key

In [ ]:
def create_augmented_vectorstore_from_json(
    original_index_path,
    output_index_path,
    wiki_chunks,
    embedding_model=OpenAIEmbeddings(model="text-embedding-3-large")
):
    # 1. Copy the original index directory
    if os.path.exists(output_index_path):
        raise FileExistsError(f"{output_index_path} already exists. Choose a new path.")
    shutil.copytree(original_index_path, output_index_path)

    # 2. Load the copied index as a new vectorstore
    vectorstore = FAISS.load_local(output_index_path, embeddings=embedding_model, allow_dangerous_deserialization=True)

    # 3. Add new documents to the vectorstore
    vectorstore.add_documents(wiki_chunks)
    vectorstore.save_local(output_index_path)

    print(f"✅ Augmented vectorstore created at {output_index_path} with {len(wiki_chunks)} wiki chunks.")


In [51]:
create_augmented_vectorstore_from_json("./faiss_index_openai_3textlarge", "./faiss_index_openai_3textlarge_full", chunks)

✅ Augmented vectorstore created at ./faiss_index_openai_3textlarge_full with 111 wiki chunks.
